In [1]:
import re
from pathlib import Path

import pandas as pd

DATA_PATH = Path("../lab_it")  # git clone https://github.com/sudakov/lab_it

# Загрузка данных

In [2]:
passenger_transportation = pd.read_csv(
    DATA_PATH / "passenger transportation.csv", sep=";"
)
cargo_transportation = pd.read_csv(DATA_PATH / "cargo transportation.csv", sep=";")
airports = pd.read_csv(
    DATA_PATH / "airports.csv",
    names=["Наименование аэропорта РФ", "Свидетельство", "Юрлицо", "Тип"],
)
airlines = pd.read_csv(
    DATA_PATH / "airlines.csv",
    names=["Название", "Юридическое название", "Наименование аэропорта РФ", "Самолеты"],
)
aircraft = pd.read_csv(
    DATA_PATH / "aircraft.csv",
    sep=";",
    parse_dates=["дата действующего свидетельства о регистрации"],
)

Приводим данные к нужному формату

In [3]:
def to_numeric(text: str) -> float:
    if isinstance(text, float):
        return text
    if '*' in text:
        return None
    text = text.replace(' ', '').replace(',', '.')
    return float(text)

In [4]:
month_columns = [
    "Январь",
    "Февраль",
    "Март",
    "Апрель",
    "Май",
    "Июнь",
    "Июль",
    "Август",
    "Сентябрь",
    "Октябрь",
    "Ноябрь",
    "Декабрь",
]

In [5]:
for column in month_columns:
    passenger_transportation[column] = passenger_transportation[column].apply(to_numeric)

for column in month_columns:
    cargo_transportation[column] = cargo_transportation[column].apply(to_numeric)

Перевод строки в список

In [6]:
airlines["Самолеты"] = airlines["Самолеты"].apply(
    lambda x: [s.strip() for s in re.sub(r"\(\d+\)", "", str(x)).split(",")]
)

Вернём колонке значение\
Добавляем min_count, чтобы сохранить `NaN` значения

In [7]:
passenger_transportation["Январь - Декабрь"] = passenger_transportation[
    month_columns
].sum(axis=1, min_count=0.1)
cargo_transportation["Январь - Декабрь"] = cargo_transportation[month_columns].sum(
    axis=1, min_count=0.1
)

Смотрим на данные

In [8]:
print('passenger_transportation'); display(passenger_transportation.head(3))
print('cargo_transportation'); display(cargo_transportation.head(3))
print('airports'); display(airports.head(3))
print('airlines'); display(airlines.head(3))
print('aircraft'); display(aircraft.head(3))

passenger_transportation


,Наименование аэропорта РФ,Год периода данных,Январь,Февраль,Март,Апрель,Май,Июнь,Июль,Август,Сентябрь,Октябрь,Ноябрь,Декабрь,Январь - Декабрь
0,Абакан,2018,11851.0,10504.0,11591.0,12323.0,14314.0,19935.0,25213.0,25507.0,19295.0,NaN,NaN,NaN,150533.0
1,Айхал,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Алдан,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


cargo_transportation


,Наименование аэропорта РФ,Год периода данных,Январь,Февраль,Март,Апрель,Май,Июнь,Июль,Август,Сентябрь,Октябрь,Ноябрь,Декабрь,Январь - Декабрь
0,Абакан,2 018,63.39,60.98,69.46,88.62,83.81,64.99,49.64,70.26,87.35,NaN,NaN,NaN,638.5
1,Айхал,2 018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Алдан,2 018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


airports


,Наименование аэропорта РФ,Свидетельство,Юрлицо,Тип
0,Архангельск (Талаги),"Сертификат МАК № 061А-М, 27.09.2013-03.08.2018...",ОАО «Аэропорт Архангельск»,В
1,"Нарьян-Мар В 3 км восточнее г.Нарьян-Мар, НАО",Свидетельство о государственной регистрации и ...,ОАО «Нарьян-Марский объединенный авиаотряд»,В
2,"Васьково 12 км юго-западнее г.Архангельск, Ар...",Свидетельство о государственной регистрации и ...,ОАО «2-ой Архангельский объединенный авиаотряд»,Г


airlines


,Название,Юридическое название,Наименование аэропорта РФ,Самолеты
551,Абакан Эир,Общество с ограниченной ответственностью «АБАК...,Абакан,"[Ил-76, Ми-26, Ми-8, Ми-8МТВ]"
481,Авиа Менеджмент Груп,Акционерное общество «Авиа Менеджмент Груп»,Иваново (Южный),[Pilatus PC-12]
554,Авиакомпания «Лайт Эйр»,Закрытое акционерное общество Авиакомпания «Ла...,Уфа,"[BK-117, Robinson R-44, Ан-2, R66]"


aircraft


,Вид воздушного судна,Тип (наименование) воздушного судна,Опознавательные знаки,Серийный №,Идентификац. №,номер свидетельства о регистрации,дата действующего свидетельства о регистрации
0,самолет,Птенец-2,RA-0001A,NaN,ЕЭВС.03.1529,0001,2012-11-03
1,дельталет,Аватар,RA-0002A,NaN,ЕЭВС.03.1709,0002,2012-03-16
2,самолет,С-2 Елочка,RA-0002G,NaN,ЕЭВС.02.0126,0224,2006-02-06


# Выполнение заданий

## Задание 1
```
Постройте список уникальных типов самолетов зарегистрированных в России.
```

In [9]:
aircraft[
    (aircraft["Вид воздушного судна"] == "самолет")
    & 
    (aircraft["Опознавательные знаки"].str.startswith("RA"))
]["Тип (наименование) воздушного судна"].drop_duplicates()

0                   Птенец-2
2                 С-2 Елочка
4                   Небесный
6                    Светлый
8                       Сура
                ...         
8909               Ил-96-300
8911             Ил-96-300ПУ
8917         Ил-96-300ПУ(М1)
8921    Ил-96-400Т/IL96-400T
8923           Ил-96-400 ВПУ
Name: Тип (наименование) воздушного судна, Length: 1853, dtype: object

## Задание 2
```
Какой тип самолета имеет раннюю дату выдачи сертификата?
```

In [10]:
aircraft[
    (aircraft["Вид воздушного судна"] == "самолет")
].groupby(
    "Тип (наименование) воздушного судна"
)["дата действующего свидетельства о регистрации"].apply(min).sort_values()

Тип (наименование) воздушного судна
Ан-2            1961-07-14
Ан-24Б          1967-01-31
Ан-24РВ         1973-04-17
Ан-26           1975-06-13
Ту-154Б         1976-07-05
                   ...    
РА-46           2017-12-15
Литл-Джет-186   2017-12-18
Twin Bee        2017-12-19
58Р             2017-12-20
E 330 LXK       2017-12-22
Name: дата действующего свидетельства о регистрации, Length: 1853, dtype: datetime64[ns]

## Задание 3
```
Постройте запрос: Владелец аэропорта, Аэропорт, Пассажиропоток суммарный за 2018 год, Грузопоток суммарный за 2018 год.
```

Пассажиропоток суммарный за 2018 год\
Удаляем строки, где NaN - недостаточно информации

In [11]:
passenger_transportation_in_2018 = passenger_transportation[
    passenger_transportation['Год периода данных'] == 2018
][['Наименование аэропорта РФ', 'Январь - Декабрь']].rename(
    columns={'Январь - Декабрь': 'Пассажиропоток суммарный за 2018 год'}
).dropna()
passenger_transportation_in_2018

,Наименование аэропорта РФ,Пассажиропоток суммарный за 2018 год
0,Абакан,150533.0
4,Анадырь,78826.0
5,Анапа(Витязево),1288050.0
8,Архангельск(Талаги),732453.0
9,Астрахань(Нариманово),450159.0
...,...,...
252,Экимчан,93.0
255,Южно-Сахалинск(Хомутово),812971.0
256,Якутск,707508.0
258,Ямбург,45828.0


Грузопоток суммарный за 2018 год\
Удаляем строки, где `NaN` - недостаточно информации

In [12]:
cargo_transportation_in_2018 = cargo_transportation[
    passenger_transportation['Год периода данных'] == 2018
][['Наименование аэропорта РФ', 'Январь - Декабрь']].rename(
    columns={'Январь - Декабрь': 'Грузопоток суммарный за 2018 год'}
).dropna()
cargo_transportation_in_2018

,Наименование аэропорта РФ,Грузопоток суммарный за 2018 год
0,Абакан,638.50
4,Анадырь,1492.64
5,Анапа(Витязево),585.61
8,Архангельск(Талаги),1385.60
9,Астрахань(Нариманово),576.40
...,...,...
252,Экимчан,0.30
255,Южно-Сахалинск(Хомутово),7647.27
256,Якутск,7055.46
258,Ямбург,1.66


Объединяем три датафрейма по колонке `Наименование аэропорта РФ`

In [13]:
airports_summary_in_2018 = airports[['Юрлицо', 'Наименование аэропорта РФ']]
airports_summary_in_2018 = pd.merge(
    airports_summary_in_2018, passenger_transportation_in_2018,
    how='outer',
    on='Наименование аэропорта РФ',
)
airports_summary_in_2018 = pd.merge(
    airports_summary_in_2018, cargo_transportation_in_2018,
    how='outer',
    on='Наименование аэропорта РФ',
)

airports_summary_in_2018 = airports_summary_in_2018.rename(columns={
    'Юрлицо': 'Владелец аэропорта',
    'Наименование аэропорта РФ': 'Аэропорт',
})
airports_summary_in_2018

,Владелец аэропорта,Аэропорт,Пассажиропоток суммарный за 2018 год,Грузопоток суммарный за 2018 год
0,ОАО «Аэропорт Архангельск»,Архангельск (Талаги),NaN,NaN
1,ОАО «Нарьян-Марский объединенный авиаотряд»,"Нарьян-Мар В 3 км восточнее г.Нарьян-Мар, НАО",NaN,NaN
2,ОАО «2-ой Архангельский объединенный авиаотряд»,"Васьково 12 км юго-западнее г.Архангельск, Ар...",NaN,NaN
3,ОАО «Аэросервис»,"Котлас 3 км юго-восточнее г.Котлас, Архангель...",NaN,NaN
4,ОАО «Аэропорт Лешуконское»,"Лешуконское 1,3 км юго-западнее села Лешуконс...",NaN,NaN
...,...,...,...,...
366,NaN,Шушенское,304.0,0.00
367,NaN,Экимчан,93.0,0.30
368,NaN,Южно-Сахалинск(Хомутово),812971.0,7647.27
369,NaN,Ямбург,45828.0,1.66


## Задание 4
```
Перечислите аэропорты, где пассажиропоток меньше медианы, а грузопоток больше медианы.
```

Средний пассажиропоток в год\
Удаляем строки, где `NaN` - недостаточно информации

In [14]:
mean_passenger_transportation = passenger_transportation.groupby(
    'Наименование аэропорта РФ'
)['Январь - Декабрь'].mean().to_frame(name='Средний пассажиропоток в год').reset_index().dropna()
mean_passenger_transportation

,Наименование аэропорта РФ,Средний пассажиропоток в год
0,Абакан,133057.583333
4,Анадырь,88478.000000
5,Анапа(Витязево),865096.083333
9,Архангельск(Талаги),676686.833333
10,Астрахань(Нариманово),376657.916667
...,...,...
299,Экимчан,93.000000
303,Южно-Сахалинск(Хомутово),797887.500000
304,Якутск,721579.000000
306,Ямбург,65083.416667


Средний грузопоток в год\
Удаляем строки, где `NaN` - недостаточно информации

In [15]:
mean_cargo_transportation = cargo_transportation.groupby(
    'Наименование аэропорта РФ'
)['Январь - Декабрь'].mean().to_frame(name='Средний грузопоток в год').reset_index().dropna()
mean_cargo_transportation

,Наименование аэропорта РФ,Средний грузопоток в год
0,Абакан,1051.512500
4,Анадырь,2254.249167
5,Анапа(Витязево),604.892500
9,Архангельск(Талаги),3122.608333
10,Астрахань(Нариманово),739.957500
...,...,...
299,Экимчан,0.300000
303,Южно-Сахалинск(Хомутово),12440.846667
304,Якутск,11981.881667
306,Ямбург,41.307500


Объединяем три датафрейма по колонке `Наименование аэропорта РФ`

In [16]:
selected_airports = airports[['Наименование аэропорта РФ']]

selected_airports = pd.merge(
    selected_airports, mean_passenger_transportation,
    how='inner',
    on='Наименование аэропорта РФ',
)
selected_airports = pd.merge(
    selected_airports, mean_cargo_transportation,
    how='inner',
    on='Наименование аэропорта РФ',
)
selected_airports

,Наименование аэропорта РФ,Средний пассажиропоток в год,Средний грузопоток в год
0,Ленск,64691.416667,948.642500
1,Мирный,314904.333333,15921.980833
2,Полярный,54989.916667,2267.197500
3,Тикси,16722.166667,326.774167
4,Якутск,721579.000000,11981.881667
5,Абакан,133057.583333,1051.512500


Запрос на выборку данных по условию

In [17]:
passenger_transportation_median = selected_airports['Средний пассажиропоток в год'].median()
print('Медиана пассажиропотока в год', passenger_transportation_median)

cargo_transportation_median = selected_airports['Средний грузопоток в год'].median()
print('Медиана грузопотока в год', cargo_transportation_median)

selected_airports[
    (selected_airports['Средний пассажиропоток в год'] < passenger_transportation_median)
    &
    (selected_airports['Средний грузопоток в год'] > cargo_transportation_median)
]

Медиана пассажиропотока в год 98874.5
Медиана грузопотока в год 1659.355


,Наименование аэропорта РФ,Средний пассажиропоток в год,Средний грузопоток в год
2,Полярный,54989.916667,2267.1975


## Задание 5
```
Перечислите авиакомпании, у которых нет типов воздушных судов зарегестрированных в России.
```

In [18]:
russian_aircraft_names = aircraft[
    aircraft["Опознавательные знаки"].str.startswith("RA")
]["Тип (наименование) воздушного судна"].unique()

airlines[
    ~airlines['Самолеты'].apply(lambda x: bool(set(x) & set(russian_aircraft_names)))
]

,Название,Юридическое название,Наименование аэропорта РФ,Самолеты
481,Авиа Менеджмент Груп,Акционерное общество «Авиа Менеджмент Груп»,Иваново (Южный),[Pilatus PC-12]
486,Аврора,Открытое акционерное общество «Авиакомпания «А...,Владивосток(Кневичи) Южно-Сахалинск (Хомутово),"[A-319, B-737, DHC-6, DHC-8]"
533,Ай Флай,ООО «Ай Флай»,Москва (Внуково),"[A-330, B-757]"
499,АТРАН,Общество с ограниченной ответственностью «АТРАН»,Москва (Домодедово) Москва (Шереметьево),[B-737]
505,Аэролимузин,Общество с ограниченной ответственностью «Аэро...,Москва (Домодедово),"[BAe-125-800, HS-125-700]"
1,Аэрофлот ОАО,ОАО «Аэрофлот - российские авиалинии»,Владивосток(Кневичи) Магадан(Сокол) Москва (...,"[A-319, A-320, A-321, A-330, B-737, RRJ-95, B-..."
451,ВИМ-АВИА,Общество с ограниченной ответственностью «Ави...,Москва (Домодедово),"[A-319, B-757]"
491,Джет Эйр Групп,ОАО «Джет Эйр Групп»,Москва (Шереметьево),"[EMB-135, Gulfstream G-IV, HS-125-700]"
89,ДОНАВИА,Открытое акционерное общество «ДОНАВИА»,Минеральные Воды Москва (Шереметьево) Ростов...,[A-319]
36,Икар ООО,Общество с ограниченной ответственностью «Авиа...,Красноярск (Емельяново),[B-767]


## Задание 6
```
Выведите список: Месяц, суммарный пассажиропоток за данный месяц, аэропорт в котором пассажиропоток в данном месяце максимальный.
```

Cуммарный пассажиропоток за данный месяц

In [19]:
passenger_transportation_sum = passenger_transportation[month_columns].sum()
passenger_transportation_sum

Январь       92581956.0
Февраль      84279197.0
Март         97548315.0
Апрель      103999255.0
Май         120740351.0
Июнь        146081971.0
Июль        167043305.0
Август      171378013.0
Сентябрь    148372441.0
Октябрь     108301174.0
Ноябрь       91962821.0
Декабрь      91356375.0
dtype: float64

Аэропорт в котором пассажиропоток в данном месяце максимальный.

In [20]:
airports_max = passenger_transportation.iloc[
    passenger_transportation[month_columns].idxmax().values
]['Наименование аэропорта РФ']
airports_max

107    Москва(Шереметьево)
107    Москва(Шереметьево)
107    Москва(Шереметьево)
107    Москва(Шереметьево)
107    Москва(Шереметьево)
107    Москва(Шереметьево)
107    Москва(Шереметьево)
107    Москва(Шереметьево)
107    Москва(Шереметьево)
388    Москва(Шереметьево)
388    Москва(Шереметьево)
388    Москва(Шереметьево)
Name: Наименование аэропорта РФ, dtype: object

Объединяем в датафрейм

In [21]:
pd.DataFrame.from_dict({
    'Месяц': month_columns,
    'Cуммарный пассажиропоток за данный месяц': passenger_transportation_sum.values,
    'Аэропорт, в котором пассажиропоток в данном месяце максимальный': airports_max.values
})

,Месяц,Cуммарный пассажиропоток за данный месяц,"Аэропорт, в котором пассажиропоток в данном месяце максимальный"
0,Январь,92581956.0,Москва(Шереметьево)
1,Февраль,84279197.0,Москва(Шереметьево)
2,Март,97548315.0,Москва(Шереметьево)
3,Апрель,103999255.0,Москва(Шереметьево)
4,Май,120740351.0,Москва(Шереметьево)
5,Июнь,146081971.0,Москва(Шереметьево)
6,Июль,167043305.0,Москва(Шереметьево)
7,Август,171378013.0,Москва(Шереметьево)
8,Сентябрь,148372441.0,Москва(Шереметьево)
9,Октябрь,108301174.0,Москва(Шереметьево)


## Задание 7
```
Выведите список: Тип аэропорта, средний грузопоток в месяц в аэропортах данного типа.
```

In [22]:
mean_cargo_transportation_in_month = cargo_transportation.groupby(
    'Наименование аэропорта РФ'
)[month_columns].mean().mean(axis=1).to_frame(name='Средний грузопоток в месяц').reset_index().dropna()
mean_cargo_transportation_in_month

,Наименование аэропорта РФ,Средний грузопоток в месяц
0,Абакан,89.659539
4,Анадырь,192.135158
5,Анапа(Витязево),51.367229
9,Архангельск(Талаги),267.807058
10,Астрахань(Нариманово),63.186149
...,...,...
299,Экимчан,0.150000
303,Южно-Сахалинск(Хомутово),1062.530833
304,Якутск,1028.509539
306,Ямбург,3.536907


Объединяем датафреймы

In [23]:
airports_cargo = airports[['Наименование аэропорта РФ', 'Тип']]

airports_cargo = pd.merge(
    airports_cargo, mean_cargo_transportation_in_month,
    how='inner',
    on='Наименование аэропорта РФ',
)
airports_cargo

,Наименование аэропорта РФ,Тип,Средний грузопоток в месяц
0,Ленск,Г,80.944280
1,Мирный,В,1362.069135
2,Полярный,Б,194.048914
3,Тикси,Г,28.035676
4,Якутск,Б,1028.509539
5,Абакан,Б,89.659539


Полученный список

In [24]:
airports_cargo.groupby('Тип')['Средний грузопоток в месяц'].mean()

Тип
Б     437.405997
В    1362.069135
Г      54.489978
Name: Средний грузопоток в месяц, dtype: float64